<a href="https://colab.research.google.com/github/antonellagambarte/procesamiento_del_lenguaje_I/blob/main/Desafio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import platform
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

from gensim.models.callbacks import CallbackAny2Vec

In [18]:
# Descarga de carpeta de dataset
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [19]:
os.listdir("./songs_dataset/")

['beatles.txt',
 'amy-winehouse.txt',
 'lady-gaga.txt',
 'lorde.txt',
 'bruce-springsteen.txt',
 'blink-182.txt',
 'dickinson.txt',
 'al-green.txt',
 'bjork.txt',
 'cake.txt',
 'bruno-mars.txt',
 'nirvana.txt',
 'bieber.txt',
 'lil-wayne.txt',
 'rihanna.txt',
 'lin-manuel-miranda.txt',
 'missy-elliott.txt',
 'janisjoplin.txt',
 'radiohead.txt',
 'britney-spears.txt',
 'leonard-cohen.txt',
 'kanye.txt',
 'kanye-west.txt',
 'Lil_Wayne.txt',
 'notorious-big.txt',
 'bob-marley.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'bob-dylan.txt',
 'ludacris.txt',
 'Kanye_West.txt',
 'adele.txt',
 'patti-smith.txt',
 'prince.txt',
 'nickelback.txt',
 'dj-khaled.txt',
 'r-kelly.txt',
 'dr-seuss.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'michael-jackson.txt',
 'nicki-minaj.txt',
 'drake.txt',
 'disney.txt',
 'notorious_big.txt',
 'paul-simon.txt',
 'nursery_rhymes.txt',
 'eminem.txt',
 'dolly-parton.txt']

In [20]:
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-2822025462.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)


,0
0,[Spoken Intro:]
1,You ever want something
2,that you know you shouldn't have
3,"The more you know you shouldn't have it,"
4,The more you want it


In [21]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9484


Preprocesamiento:

In [22]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))


sentence_tokens[:2]

[['spoken', 'intro'], ['you', 'ever', 'want', 'something']]

In [23]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [24]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [25]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [26]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9484


In [27]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1000
